In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import pymysql
import os
import base64
import json
import logging
import time
import csv
import datetime
from datetime import timedelta

MAX_NB_WORDS = 100000
# SNU_model = load_model('SNU_LSTM_Model.h5')
SNU_model = load_model('SNU_LSTM_Model_shuffle.h5')

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-08-10 10:13:37.399 ip-172-31-2-138:1660 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-10 10:13:37.420 ip-172-31-2-138:1660 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


**MySQL 데이터베이스 저장**

In [54]:
host = '' # rds endpoint
port = 3306
username = ''
database = ''
password = ''

def connect_RDS(host, port, username, password, database):
    try:
        conn = pymysql.connect(host = host, user = username, password = password, db = database, port = port, use_unicode = True, charset = 'utf8')
        cursor = conn.cursor()
    
    except:
        logging.error('RDS에 연결되지 않았습니다.')
        sys.exit(1)
        
    return conn, cursor

conn, cursor = connect_RDS(host, port, username, password, database)

In [2]:
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

,document,label,doc_len
0,현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다,1,9
1,"‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다",1,8
2,액상형 전자담배는 니코틴이 함유되지 않아 담배가 아니다,1,7
3,한국 백신 접종률 세계 100위권 이하다,1,6
4,광역도시 없는 지역권은 실제 수요와 관계없이 획일적으로 국가교통망 정책에서 소외되고 있다,1,11
...,...,...,...
1219,""" ‘당·정·청 전원회의’는 '운동권 용어'다""?",0,5
1220,"""늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)""?",0,6
1221,"""청와대 하루 평균 업무추진비, 이명박 768만원, 박근혜 814만원, 문재인 55만원""",0,10
1222,"국민연금공단 기금운용본부가 전주로 이전한 후, 해외투자자가 국민연금을 '패싱'한다",0,8


**단어 벡터화 및 패딩**

In [3]:
raw_docs_train = train_df['document'].tolist()
num_classes = len(label_names)
print(num_classes)
processed_docs_train = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_index = tokenizer.word_index
print('dictionary size :', len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train

  0%|          | 0/1224 [00:00<?, ?it/s]

1


100%|██████████| 1224/1224 [00:05<00:00, 209.68it/s]

dictionary size : 3999


array([[ 503, 1459,  504, ..., 1461, 1462,    8],
       [   0,   72,  761, ...,   51,  360,    5],
       [   0,    0, 1466, ...,  506,  763,   41],
       ...,
       [ 649,  177,  381, ..., 3992,   13, 3993],
       [3994, 3995, 3996, ...,  248, 3998,    5],
       [   0,    0,    0, ..., 1401, 3999,  496]], dtype=int32)

In [5]:
# import os
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

**사용자에게 받은 데이터 전처리 테스트**

In [4]:
response = ['현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다', 'ajsdlkfakljflalejlka', 'dshkjlafjsdkfjalks']
tokens_response = []

for i in range(len(response)):
    tokens = my_Tokenizer(response[i])
    tokens_response.append(tokens)
#     print(tokens_response)

word_seq_response = tokenizer.texts_to_sequences(tokens_response)
print(word_seq_response[0])
word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
print(word_seq_response.shape)

score_list = []
for sco in range(len(word_seq_response)):
    word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
    print(word_seq_response_to_score)
    score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
    print(score_list)

[503, 1459, 504, 1460, 22, 71, 104, 1461, 1462, 8]
(3, 10)
[[ 503 1459  504 1460   22   71  104 1461 1462    8]]


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/bidirectional/forward_lstm/PartitionedCall]] [Op:__inference_predict_function_10128]

Function call stack:
predict_function -> predict_function -> predict_function


**키워드, 기사제목 입력시 웹서버**

In [57]:
url_lambda = 'https://jub0t0f0nc.execute-api.ap-northeast-2.amazonaws.com/default/Python_jupyter_test'

class S(BaseHTTPRequestHandler):
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
#         logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
#                 str(self.path), str(self.headers), post_data.decode('utf-8'))
        
        response = str(post_data.decode('utf-8'))
        response_list = []
        response_list.append(response)
        response1 = ['카카오톡 유료화 서비스 시행하나?']
        
        
# ===========================================================================================================================
# 단어 토큰화 및 빈도순 정렬 후 추출

        tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
        X = tfidf_Vectorizer.fit_transform(response_list).toarray()
        print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
        print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


        #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
        count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
        word_count = pd.DataFrame({
            '단어' : tfidf_Vectorizer.get_feature_names(),
            '빈도' : count.flat
        })
        sorted_df = word_count.sort_values('빈도', ascending=False)
        print(sorted_df.head(10), "\n")

        word_ = list(np.array(sorted_df['단어'].tolist())) 
        
# ===========================================================================================================================
# MySQL 데이터베이스에 응답데이터, 시간 저장
        
        global cursor, conn
        conn, cursor = connect_RDS(host, port, username, password, database)
        
        now = datetime.datetime.now()
        Korea_DatetimeNow = now + datetime.timedelta(hours = 9)
        DatetimeNow = Korea_DatetimeNow.strftime('%Y-%m-%d %H:%M:%S')
        
        query = "INSERT INTO factcheck.article_data (article_data, date) VALUES(%s, %s)"
        val = (response, DatetimeNow)
        cursor.execute(query, val)
        conn.commit()
        
# ===========================================================================================================================
# MySQL 데이터베이스에 키워드 랭킹

        rank_list = []

        cursor.execute('SELECT keyword FROM keyword_rank')
        row = cursor.fetchall()

        for i in range(len(row)):
            rank_list.append(row[i][0])
            
        for i in range(0,2):
            if word_[i] in rank_list:
                query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
                val = (word_[i])
                cursor.execute(query, val)
                conn.commit()
            else:
                query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
                val = (word_[i])
                cursor.execute(query, val)
                conn.commit()

            conn.close()
        
        top10_list = []
        cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
        row_top = cursor.fetchall()
        for i in range(len(row_top)):
            top10_list.append(row_top[i][0])
        
# ===========================================================================================================================
# SNU 기사 검색      

        try:
            str_expr = "document.str.contains('{}', case=False)".format(word_[0]) and "document.str.contains('{}', case=False)".format(word_[1])
            df_q = train_df.query(str_expr, engine="python")

            title = list(np.array(df_q['document'].tolist()))
            # link = list(np.array(df_q['link'].tolist()))
            title_list2 = []
            # link_list2 = []

            for i in range(len(title)):
                title_list2.append(title[i])
            #     link_list2.append(link[i])
            
        except:
            pass
        
# ===========================================================================================================================
# 네이버 뉴스 검색

        baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

        url = baseurl + urllib.parse.quote_plus(response)

        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.find_all(class_ = 'news_tit')

        title_list = []
        link_list = []

        for i in title:
            title_list.append(i.attrs['title'])
            link_list.append(i.attrs['href'])

# ===========================================================================================================================

        tokens_response = []
        tokens_response2 = []
        score_list = []
        score_list2 = []
        
        for i in range(len(title_list)):
            tokens = my_Tokenizer(title_list[i])
            tokens_response.append(tokens)

        word_seq_response = tokenizer.texts_to_sequences(tokens_response)
        word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
        
        for sco in range(len(title_list)):
            word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
            score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
        
        try:
            for i in range(len(title_list2)):
                tokens2 = my_Tokenizer(title_list2[i])
                tokens_response2.append(tokens2)

            word_seq_response2 = tokenizer.texts_to_sequences(tokens_response2)
            word_seq_response2 = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

            for sco2 in range(len(title_list2)):
                word_seq_response_to_score2 = word_seq_response2[sco2].reshape(1,max_seq_len)
                score_list2.append(float(SNU_model.predict(word_seq_response_to_score2)))
        
        except:
            pass
        
        data_ = {
            'response':response,
            'Naver_title':title_list,
            'Naver_link':link_list,
            'Naver_score':score_list,
            'SNU_title':title_list2,
            'SNU_score':score_list2,
            'top10_keyword':top10_list
            }
        
#         requests.post(url_lambda, data = response.encode())
        requests.post(url_lambda, data = json.dumps(data_))
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('네이버 뉴스 제목 : \n', title_list)
        print('\n')
        print('네이버 뉴스 링크 : \n', link_list)
        print('\n')
        print('네이버 뉴스 score : \n', score_list)
        print('\n')
        try:
            print('SNU 기사 제목 : \n', title_list2)
            print('\n')
    #         print('SNU 기사 링크 : ', link_list2)
            print('SNU 기사 score : \n', score_list2)
        except:
            print('SNU 기사가 없습니다. \n')

        self._set_response()
        self.wfile.write("POST request for {}".format(self.path).encode('utf-8'))

def run(server_class=HTTPServer, handler_class=S, port=8080):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    logging.info('Starting httpd...\n')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    logging.info('Stopping httpd...\n')

if __name__ == '__main__':
    from sys import argv

    if len(argv) == 2:
        run(port=int(argv[1]))
    else:
        run()

INFO:root:Starting httpd...



(1, 2)
{'카카오': 1, '유료': 0}
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 

입력 데이터 : 
 카카오톡 유료화


네이버 뉴스 제목 : 
 ['카카오모빌리티 잇따른 요금인상..."수금본색"vs"시장논리"', "넷플릭스식 '네이버' 유튜브식 '카카오'...같은듯 다른 인터넷 양강의 구독 실험", '대리운전연합 "카카오·SKT, 전화콜마저 진출…골목시장 침탈"', '“카카오·티맵, 대리콜까지 빼앗냐”']


네이버 뉴스 링크 : 
 ['https://zdnet.co.kr/view/?no=20210809105033', 'https://www.techm.kr/news/articleView.html?idxno=86902', 'https://news.joins.com/article/olink/23716744', 'https://news.joins.com/article/olink/23716962']


네이버 뉴스 score : 
 [0.5055016875267029, 0.5024815797805786, 0.5055860280990601, 0.5053588151931763]


SNU 기사 제목 : 
 ['"카카오페이 얼굴인식, 중국으로 개인정보가 넘어간다"']


SNU 기사 score : 
 [0.5055016875267029]
(1, 2)
{'카카오': 1, '유료': 0}
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 



3.36.48.61 - - [09/Aug/2021 05:51:04] "POST / HTTP/1.1" 200 -


입력 데이터 : 
 카카오톡 유료화


네이버 뉴스 제목 : 
 ['카카오모빌리티 잇따른 요금인상..."수금본색"vs"시장논리"', "넷플릭스식 '네이버' 유튜브식 '카카오'...같은듯 다른 인터넷 양강의 구독 실험", '대리운전연합 "카카오·SKT, 전화콜마저 진출…골목시장 침탈"', '“카카오·티맵, 대리콜까지 빼앗냐”']


네이버 뉴스 링크 : 
 ['https://zdnet.co.kr/view/?no=20210809105033', 'https://www.techm.kr/news/articleView.html?idxno=86902', 'https://news.joins.com/article/olink/23716744', 'https://news.joins.com/article/olink/23716962']


네이버 뉴스 score : 
 [0.5055016875267029, 0.5024815797805786, 0.5055860280990601, 0.5053588151931763]


SNU 기사 제목 : 
 ['"카카오페이 얼굴인식, 중국으로 개인정보가 넘어간다"']


SNU 기사 score : 
 [0.5055016875267029]
(1, 2)
{'카카오': 1, '유료': 0}
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 



13.209.10.79 - - [09/Aug/2021 05:51:08] "POST / HTTP/1.1" 200 -


입력 데이터 : 
 카카오톡 유료화


네이버 뉴스 제목 : 
 ['카카오모빌리티 잇따른 요금인상..."수금본색"vs"시장논리"', "넷플릭스식 '네이버' 유튜브식 '카카오'...같은듯 다른 인터넷 양강의 구독 실험", '대리운전연합 "카카오·SKT, 전화콜마저 진출…골목시장 침탈"', '“카카오·티맵, 대리콜까지 빼앗냐”']


네이버 뉴스 링크 : 
 ['https://zdnet.co.kr/view/?no=20210809105033', 'https://www.techm.kr/news/articleView.html?idxno=86902', 'https://news.joins.com/article/olink/23716744', 'https://news.joins.com/article/olink/23716962']


네이버 뉴스 score : 
 [0.5055016875267029, 0.5024815797805786, 0.5055860280990601, 0.5053588151931763]


SNU 기사 제목 : 
 ['"카카오페이 얼굴인식, 중국으로 개인정보가 넘어간다"']


SNU 기사 score : 
 [0.5055016875267029]
(1, 2)
{'카카오': 1, '유료': 0}
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 



13.209.10.79 - - [09/Aug/2021 05:51:11] "POST / HTTP/1.1" 200 -
INFO:root:Stopping httpd...



In [59]:
conn, cursor = connect_RDS(host, port, username, password, database)
response = '유후~'
rank_list = []

cursor.execute('SELECT keyword FROM keyword_rank')
row = cursor.fetchall()

for i in range(len(row)):
    rank_list.append(row[i][0])
    
if response in rank_list:
    query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
    val = (response)
    cursor.execute(query, val)
    conn.commit()
else:
    query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
    val = (response)
    cursor.execute(query, val)
    conn.commit()
    
conn.close()

In [72]:
conn, cursor = connect_RDS(host, port, username, password, database)
top10_list = []
cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
# cursor.execute("SELECT keyword FROM keyword_rank LIMIT 4")
row = cursor.fetchall()
for i in range(len(row)):
    top10_list.append(row[i][0])

top10_list

['카카오톡 유료화', '성공했군', '성공했나', '유후~']

In [60]:
# import codecs
# from tqdm import tqdm
# import numpy as np

# embeddings_index = {}
# f = codecs.open('/home/ubuntu/FastText/wiki.ko.vec', encoding='utf-8')

# for line in tqdm(f):
#     values = line.rstrip().rsplit(' ')
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('found %s word vectors' % len(embeddings_index))

In [6]:
# import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)
    
# SNU_model = load_model('SNU_LSTM_Model.h5')


In [46]:
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = ['카카오톡 유료화 서비스 시행하나?']

tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
X = tfidf_Vectorizer.fit_transform(sentences).toarray()
print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


#pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
word_count = pd.DataFrame({
    '단어' : tfidf_Vectorizer.get_feature_names(),
    '빈도' : count.flat
})
sorted_df = word_count.sort_values('빈도', ascending=False)
print(sorted_df.head(10), "\n")

word_ = list(np.array(sorted_df['단어'].tolist())) 

(1, 5)
{'카카오': 3, '유료': 2, '서비스': 0, '시행': 1, '하나': 4}
    단어        빈도
0  서비스  0.447214
1   시행  0.447214
2   유료  0.447214
3  카카오  0.447214
4   하나  0.447214 



In [21]:
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="
response = ['카카오톡 유료화 서비스 시행하나?']

url = baseurl + urllib.parse.quote_plus(response[0])

html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all(class_ = 'news_tit')

title_list = []
link_list = []

for i in title:
    print(i.attrs['title'])
    print(i.attrs['href'])
    print()
    title_list.append(i.attrs['title'])
    link_list.append(i.attrs['href'])
    
print(title_list)
print(link_list)

카카오式 문어발 확장에 진통…"불공정 유료화 국토부와 논의중"
http://www.dizzotv.com/site/data/html_dir/2021/04/22/2021042280184.html

카카오T 유료화에 뿔난 택시업계... "불공정 배차 조사해야"
http://biz.newdaily.co.kr/site/data/html/2021/04/27/2021042700001.html

서민의 '프렌즈' 되랬더니, 고신용 고객만 쓸어담은 카카오뱅크
http://news.mt.co.kr/mtview.php?no=2021041601224584614

카카오 쏟아지는 이슈…주가 어디까지(종합)
http://www.smedaily.co.kr/news/articleView.html?idxno=201337

['카카오式 문어발 확장에 진통…"불공정 유료화 국토부와 논의중"', '카카오T 유료화에 뿔난 택시업계... "불공정 배차 조사해야"', "서민의 '프렌즈' 되랬더니, 고신용 고객만 쓸어담은 카카오뱅크", '카카오 쏟아지는 이슈…주가 어디까지(종합)']
['http://www.dizzotv.com/site/data/html_dir/2021/04/22/2021042280184.html', 'http://biz.newdaily.co.kr/site/data/html/2021/04/27/2021042700001.html', 'http://news.mt.co.kr/mtview.php?no=2021041601224584614', 'http://www.smedaily.co.kr/news/articleView.html?idxno=201337']


In [25]:
str_expr = "document.str.contains('{}', case=False)".format(word_[3]) and "document.str.contains('{}', case=False)".format(word_[2])
df_q = train_df.query(str_expr, engine="python")
df_q

,document,label,doc_len
293,"""유료광고 속인 인플루언서, 제재 못 한다""",1,6


In [ ]:
title = list(np.array(df_q['document'].tolist()))
# link = list(np.array(df_q['link'].tolist()))
title_list2 = []
# link_list2 = []

for i in range(len(title)):
    title_list2.append(title[i])
#     link_list2.append(link[i])